In [11]:
import os
import json
import math
import numpy as np
from scipy.spatial.transform import Rotation

In [12]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_one_spin')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [13]:
transforms = {}
transforms["focal_length"] = 50.0 #Focal Length in mm
transforms["fov_x"] = 1.05 #FOV in radians
transforms["w"] = 1280 # image width
transforms["h"] = 720 # image height


transforms["pixel_width"] = 2*math.tan(transforms["fov_x"]) * transforms["focal_length"] / transforms["w"] #Sensor width in mm
transforms["pixel_height"] = transforms["pixel_width"] #Sensor height in mm
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

In [14]:
frames = []

starting_pts = [[0.5,0]]
z_store = [1]
pitch = 0.7
spin = 5

cnt = 0

for x, y in starting_pts:
    for z in z_store:
        start_transform_matrix = np.eye(4)
        start_transform_matrix[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, 0])).as_matrix()
        start_transform_matrix[0, 3] = x
        start_transform_matrix[1, 3] = y
        start_transform_matrix[2, 3] = z

        for i in range(spin):
            angle = i * 2 * np.pi / spin
            rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
            rot_matrix = np.eye(4)
            rot_matrix[:3, :3] = rot
            transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

            frame = {}
            file_path = "images/{}.png".format(cnt)

            frame["file_path"] = file_path
            frame["transform_matrix"] = transform_matrix.tolist()

            frames.append(frame)

            print(transform_matrix[2, 3])

            cnt += 1


transforms["frames"] = frames

1.0
1.0
1.0000000000000002
1.0
1.0


In [15]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)